In [1]:
import os

In [2]:
%pwd

'c:\\SkinDiseaseProject\\skin_disease_predictor\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\SkinDiseaseProject\\skin_disease_predictor'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file:str
    unzip_dir:Path   


In [6]:
from Acne_Prediction.constants import *
from Acne_Prediction.utils.common import read_yaml, create_directories


In [7]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])    
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
import gdown
from Acne_Prediction import logger
from Acne_Prediction.utils.common import get_size 

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        """
        Downloads the file from Google Drive or any URL and saves it locally.
        Uses gdown for Google Drive links.
        """
        if not os.path.exists(self.config.local_data_file):
            # Extract file id from Google Drive link
            file_id = self.config.source_URL.split('/d/')[1].split('/')[0]
            download_url = f"https://drive.google.com/uc?id={file_id}"
            gdown.download(download_url, self.config.local_data_file, quiet=False)
            logger.info(f"Downloaded file to {self.config.local_data_file}")
        else:
            logger.info(f"{self.config.local_data_file} already exists of size {get_size(self.config.local_data_file)}")
    def extract_zip_file(self):
        """
        zip_file_path :str
        Extracts the zip file at zip_file_path to the destination directory
        Function return none
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted zip file to {unzip_path}")
        
        

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-21 21:02:17,522: Successfully loaded configuration from config\config.yaml: common: Successfully loaded configuration from config\config.yaml]
[2025-04-21 21:02:17,523: Successfully loaded configuration from params.yaml: common: Successfully loaded configuration from params.yaml]
[2025-04-21 21:02:17,523: created directory at artifacts: common: created directory at artifacts]
[2025-04-21 21:02:17,524: created directory at artifacts/data_ingestion: common: created directory at artifacts/data_ingestion]


Downloading...
From (original): https://drive.google.com/uc?id=1wsjksh4NHOgVX2vwcClBZTSyz8N16lb-
From (redirected): https://drive.google.com/uc?id=1wsjksh4NHOgVX2vwcClBZTSyz8N16lb-&confirm=t&uuid=68704748-d1d6-48ea-98c1-88b5d295b3f7
To: c:\SkinDiseaseProject\skin_disease_predictor\artifacts\data_ingestion\4classData.zip
100%|██████████| 1.08G/1.08G [05:26<00:00, 3.31MB/s]

[2025-04-21 21:07:49,165: Downloaded file to artifacts/data_ingestion/4classData.zip: 2236992415: Downloaded file to artifacts/data_ingestion/4classData.zip]


[2025-04-21 21:07:56,925: Extracted zip file to artifacts/data_ingestion: 2236992415: Extracted zip file to artifacts/data_ingestion]
